In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Install the ultralytics package using pip
!pip install ultralytics

In [ ]:
import os
working_dir = '/kaggle/working/'
yolo_data = '/kaggle/working/yolo_data'
if yolo_data not in working_dir:
    os.makedirs(yolo_data)
    print('directory created succsefullly')
else:
    print('not logic failed')

In [ ]:
yolo_data = '/kaggle/working/yolo_data'
training_data = '/kaggle/working/yolo_data/train'
val_data = '/kaggle/working/yolo_data/val'
if training_data not in yolo_data:
    os.makedirs(training_data)
    print('image directory created succsefullly')
else:
    print('image directory not created')
    
if val_data not in yolo_data:
    os.makedirs(val_data)
    print('label directory created')
else :
    print('label directory failed to create')

In [ ]:
import shutil
# copy all training img into working directory 
for file in os.listdir('/kaggle/input/business-registration-certificate-ocr/train_image/train_image'):
    source_path = '/kaggle/input/business-registration-certificate-ocr/train_image/train_image'+'/'+file
    dest_path = '/kaggle/working/yolo_data/train'+'/'+file
    shutil.copy(source_path,dest_path)
# now lets copy all validation image 
for file in os.listdir('/kaggle/input/business-registration-certificate-ocr/test_image/test_image'):
    source_path = '/kaggle/input/business-registration-certificate-ocr/test_image/test_image'+'/'+file
    dest_path = '/kaggle/working/yolo_data/val'+'/'+file
    shutil.copy(source_path,dest_path)
# we will folloew same procedure for val label and train label 
for file in os.listdir('/kaggle/input/business-registration-certificate-ocr/train_annotation/train_annotation'):
    source_path = '/kaggle/input/business-registration-certificate-ocr/train_annotation/train_annotation'+'/'+file
    dest_path = '/kaggle/working/yolo_data/train'+'/'+file
    shutil.copy(source_path,dest_path)
for file in os.listdir('/kaggle/input/business-registration-certificate-ocr/test_annotation/test_annotation'):
    source_path = '/kaggle/input/business-registration-certificate-ocr/test_annotation/test_annotation'+'/'+file
    dest_path = '/kaggle/working/yolo_data/val'+'/'+file
    shutil.copy(source_path,dest_path)

In [ ]:
import os
import yaml

# Set the folder path where you want to create the YAML file
folder_path = "/kaggle/working/"  # Replace with the desired folder path

# Create the YAML file name and path
file_name = "dataset.yaml"  # Replace with the desired file name
file_path = os.path.join(folder_path, file_name)

# YAML content to be written in the file
yaml_content = {
    "path": '/kaggle/input/business-registration-certificate-ocr',
    "train": "/kaggle/working/yolo_data/train",
    "val": "/kaggle/working/yolo_data/val",
    'names':{0:'DOB',
             1:'DOR',
             2:'Business name',
             3:'Business add',
             4:'Business type',
             5:'Business cat',
             6:'Reg no '}
}

# Create the YAML file and write the content
with open(file_path, "w") as file:
    yaml.dump(yaml_content, file)

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
os.makedirs('/kaggle/working/training_result')
folder_path = '/kaggle/working/training_result'

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8m.yaml')  # build a new model from YAML
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8m.yaml').load('yolov8m.pt')  # build from YAML and transfer weights

# Train the model
model.train(data='/kaggle/working/dataset.yaml', epochs=50, imgsz=640,project = folder_path)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8m.pt')  # load an official model
model = YOLO('/kaggle/working/training_result/train/weights/best.pt')  # load a custom model

# Predict with the model
results = model('/kaggle/input/business-registration-certificate-ocr/test_image/test_image/2022_05_16_17_Resized_20220516_174022.jpg')  # predict on an image

import matplotlib.pyplot as plt 


# Plot the image with increased figure size
res_plotted = results[0].plot()
plt.figure(figsize=(15, 10))  # Adjust the figure size as needed
plt.imshow(res_plotted)
plt.axis('off')  # Turn off axis ticks and labels
plt.show()

In [ ]:
print(results[0].boxes.data)

In [ ]:
from ultralytics import YOLO
import cv2

# Load a model
model = YOLO('yolov8m.pt')  # Load an official model

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8m.pt') # Load an official model
model = YOLO('/kaggle/working/training_result/train/weights/best.pt') 

# Predict with the model
results = model('/kaggle/input/business-registration-certificate-ocr/test_image/test_image/2022_05_16_17_Resized_20220516_174022.jpg') 

# Load the original image
original_image = cv2.imread('/kaggle/input/business-registration-certificate-ocr/test_image/test_image/2022_05_16_17_Resized_20220516_174022.jpg')

# Iterate through the list of results
for detection in results[0].boxes.data:
#     print(detection)
    # Convert to integer
    x, y, xmax, ymax,conf,label = detection.int().tolist()
    conf  = detection[4].int().tolist()
    # Crop the image
    cropped_image = original_image[y:ymax, x:xmax]
    # Display the cropped region using OpenCV
    # Display the cropped region using matplotlib
    print(label)
    plt.figure(figsize=(5, 5))
    plt.imshow(cropped_image)
    
    # Add text for class label
    
    plt.axis('off')
    plt.show()
    

